In [2]:
import os
from datetime import datetime
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import smtplib
from tqdm import tqdm
from typing import Optional

In [3]:
load_dotenv()

True

In [4]:
sender_email = os.getenv("SENDER_EMAIL_ADDRESS")
sender_password = os.getenv("SENDER_EMAIL_PASSWORD")

In [5]:
class OutlookEmailClient:
    def __init__(
        self,
        sender_email: str,
        sender_password: str,
        smtp_server: str = "smtp-mail.outlook.com",
        smtp_port: int = 587,
    ) -> None:
        self.sender_email = sender_email
        self.sender_password = sender_password
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port

    def __enter__(self):
        self.connection = self._create_connection()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb) -> None:
        self.connection.close()

    def _create_connection(self) -> smtplib.SMTP:
        server = smtplib.SMTP(self.smtp_server, self.smtp_port)
        server.starttls()
        server.login(self.sender_email, self.sender_password)
        return server

    def send_email(
        self,
        recipient_email: str,
        cc_recipients: list[str],
        subject: str,
        body: str,
        attachments: Optional[list[str]] = None,
    ) -> None:
        """
        Send a single email with optional attachments.

        Args:
            recipient_email (str): Email address of the recipient
            subject (str): Email subject line
            body (str): Email body content
            attachments (list[str], optional): List of file paths to attach
        """
        msg = MIMEMultipart()
        msg["From"] = self.sender_email
        msg["To"] = recipient_email
        msg["Cc"] = ", ".join(cc_recipients) if cc_recipients else ""
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "html"))

        if attachments:
            for file_path in attachments:
                with open(file_path, "rb") as f:
                    attachment = MIMEApplication(f.read(), _subtype="xlsx")
                    filename = file_path.split("/")[-1]
                    attachment.add_header(
                        "Content-Disposition", "attachment", filename=filename
                    )
                    msg.attach(attachment)

        self.connection.send_message(msg)

In [6]:
emails = [
    "alex.soldin@gmail.com",
]
# emails = [
#     "alex.soldin@gmail.com",
#     "natasha.soldin@gmail.com",
#     "larry@sunriselogistics.net",
# ]

In [7]:
with OutlookEmailClient(
    sender_email=sender_email, sender_password=sender_password
) as client:
    for email in tqdm(emails):
        client.send_email(
            recipient_email=email,
            cc_recipients=[],
            subject=f"Frequency Report {datetime.now().strftime('%d %b %Y')}",
            body="""
            <html>
            <head>
                <style>
                    body {
                        font-family: Arial, sans-serif;
                        font-size: 14px;
                        line-height: 1.6;
                        color: #333333;
                    }
                </style>
            </head>
            <body>
            <p>Dear Recipient,</p>

            <p>I hope this email finds you well. Please find attached your Frequency Report for review.</p>

            <p>This automated report contains:</p>
            <ul>
                <li>Current deliveries</li>
                <li>Completed deliveries</li>
            </ul>

            <p>For additional information, please visit our <a href="https://www.sunriselogistics.net/">website</a>.</p>

            <p>If you have any questions or need clarification about the contents of this report, please don't hesitate to reach out.</p>

            <p>Best regards,<br>
            </body>
            </html>
            """,
            attachments=["../assets/frequency_report_template.xlsx"],
        )

100%|██████████| 1/1 [00:02<00:00,  2.29s/it]
